#### Import the required Libraries 

In [ ]:
#Data handling packages
import pandas as pd

#Data processing packages
import numpy as np

#Visualization packages
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Additional libraries 
import re
import string
import time
import os

#NLP based libraries
from wordcloud import WordCloud, STOPWORDS
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
import googletrans
from googletrans import Translator


#Preprocessing and model building libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.feature_selection import chi2
from sklearn.ensemble import GradientBoostingClassifier


#framework 
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
data= pd.read_csv("C:\\Users\\Riyaz Mehendi\\OneDrive - CriticalRiver Technologies Pvt. Ltd\\Desktop\\MVP Incident Management\\pim-assignment-group-prediction\\data\\processed\\final_processed_data_1_EBS.csv")

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data=data.drop(['Opened','Resolved',
       'Business resolve time', 'name_string_validation', 'langue',
       'translated_text','string_length'], axis = 1)

In [ ]:
data

In [ ]:
data = data.replace(['Desktop Services UK', 'Desktop Services Brazil', 'Desktop Services USA Central','Desktop Services India','Desktop Services USA West','Desktop Services Germany','Desktop Services Singapore','Desktop Services USA East',
 'Desktop Services Mexico','Desktop Services China','Desktop Services Canada','Desktop Services France','Desktop Services Romania','Desktop Services Thailand'], 'Desktop Services')


In [ ]:
data

In [ ]:
target_names = data['Assignment group'].unique()

In [ ]:
type(target_names)

In [ ]:
target_names=list(target_names)

In [ ]:
target_names

In [ ]:
len(target_names)

In [ ]:
data.isnull().sum()

In [ ]:
data = data.replace(np.nan, 'Others')

In [ ]:
data.isnull().sum()

In [ ]:
data['cleaned_name']=data['cleaned_name'].astype(str)

Encoding the categorical variables

In [ ]:
def EncodeSingleColumn(data):
    le = preprocessing.LabelEncoder()
    le.fit(data)
    return le.transform(data)

In [ ]:
def EncodeColumnsCollection(data, columnsToEncrypt):
    for column in columnsToEncrypt:
        data[column] = EncodeSingleColumn(data[column])
    return data

In [ ]:
# Select columns for encryption
columnsToEncode = ['Priority','Urgency','Configuration item','Subcategory','Category']

# Encrypt data for each of selected columns
df_encoded = EncodeColumnsCollection(data, columnsToEncode)

In [ ]:
df_encoded.head()

In [ ]:
encoded_data=df_encoded.drop(['Number','Name','cleaned_name'], axis = 1)

In [ ]:
encoded_data

In [ ]:
encoded_data['Assignment group'].unique()

#### Text Embedding using USE technique

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

In [ ]:
embedded_desc1 = model(data['cleaned_name'])

In [ ]:
embedded_desc1

In [ ]:
#converta array into dataframe
embedded_desc1=pd.DataFrame(embedded_desc1)
embedded_desc1.head()

In [ ]:
new_data = pd.concat([embedded_desc1, encoded_data], axis=1)

In [ ]:
new_data

Split the Dataset into Train and Test

In [ ]:
y= new_data['Assignment group']

In [ ]:
y.value_counts()

In [ ]:
X= new_data.drop(['Assignment group','Priority','Urgency','Configuration item','Subcategory','Category'], axis = 1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0, stratify=y)
print(len(x_train))
print(len(x_test))

In [ ]:
y_train.value_counts()

In [ ]:
y_train_list=y_train.tolist()

In [ ]:
y_train_list

In [ ]:
y_test.value_counts()

In [ ]:
y_test_list=y_test.tolist()

In [ ]:
y_test_list

In [ ]:
Y_List= pd.DataFrame({'Y_train': pd.Series(y_train_list).value_counts(), 'Y_test': pd.Series(y_test_list).value_counts()})

In [ ]:
Y_List

In [ ]:
Y_List.reset_index(level=0, inplace=True)

In [ ]:
#Y_List.to_csv('Assignment_group_Class.csv',index=False)

#### Model Building

In [ ]:
DT_model = DecisionTreeClassifier().fit(x_train, y_train)
y_pred = DT_model.predict(x_test)

In [ ]:
# Classification report
print('\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred))

In [ ]:
SVC_model= SVC().fit(x_train, y_train)
y_pred = SVC_model.predict(x_test)

In [ ]:
# Classification report
print('\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred))

In [ ]:
KNN_model=KNeighborsClassifier().fit(x_train, y_train)
y_pred1 = KNN_model.predict(x_test)

In [ ]:
 #print('Training accuracy: %.2f%%' % (accuracy_score(y_train,y_pred1) * 100))

In [ ]:
# Classification report
print('\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred1))

In [ ]:
RF_model= RandomForestClassifier().fit(x_train,y_train)
y_pred1 = RF_model.predict(x_test)

In [ ]:
# Classification report
print('\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred1))

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test=le.fit_transform(y_test)

In [ ]:
XGB_model= XGBClassifier().fit(x_train,y_train)
y_pred1 = XGB_model.predict(x_test)

In [ ]:
#### Classification report
print('\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred1))

With USE Embedding on 80% trained data 

Decision Tree - 67
Random Forest - 63
Gaussian Naive Bayes - 38 
SVC -  52
KNN - 78



In [ ]:
With tfidf embedding  and training on 80 as train data and replacing with desktop services for all countries and syntehsized one class EBS markview with one more sample and X used only cleaned_Data

Decision tree-  
SVC - 
KNN - 
Random Forest - 